# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv('weather_data_output.csv')

weather_data_df = pd.DataFrame(weather_data)
weather_data_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed (mph),Country,Date
0,0,Novotroitskaya,45.33,41.53,30.00,100,90,11.18,RU,11/22/2020
1,1,Bathsheba,13.22,-59.52,80.60,78,40,14.99,BB,11/22/2020
2,2,Albany,42.60,-73.97,39.00,67,100,1.99,US,11/22/2020
3,3,Saint-Philippe,-21.36,55.77,73.40,73,86,11.41,RE,11/22/2020
4,4,Conceição do Araguaia,-8.26,-49.26,73.20,94,100,1.68,BR,11/22/2020
...,...,...,...,...,...,...,...,...,...,...
555,555,Muros,42.78,-9.06,46.00,93,0,4.79,ES,11/22/2020
556,556,Pontes e Lacerda,-15.23,-59.34,86.76,36,77,3.02,BR,11/22/2020
557,557,Rawson,-43.30,-65.10,71.31,34,0,18.21,AR,11/22/2020
558,558,Sabha,27.04,14.43,62.37,31,45,12.66,LY,11/22/2020


In [3]:
del weather_data_df['Unnamed: 0']
weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed (mph),Country,Date
0,Novotroitskaya,45.33,41.53,30.00,100,90,11.18,RU,11/22/2020
1,Bathsheba,13.22,-59.52,80.60,78,40,14.99,BB,11/22/2020
2,Albany,42.60,-73.97,39.00,67,100,1.99,US,11/22/2020
3,Saint-Philippe,-21.36,55.77,73.40,73,86,11.41,RE,11/22/2020
4,Conceição do Araguaia,-8.26,-49.26,73.20,94,100,1.68,BR,11/22/2020
...,...,...,...,...,...,...,...,...,...
555,Muros,42.78,-9.06,46.00,93,0,4.79,ES,11/22/2020
556,Pontes e Lacerda,-15.23,-59.34,86.76,36,77,3.02,BR,11/22/2020
557,Rawson,-43.30,-65.10,71.31,34,0,18.21,AR,11/22/2020
558,Sabha,27.04,14.43,62.37,31,45,12.66,LY,11/22/2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key) 

# Store latitude and longitude in locations, and humidity as humidity
locations = weather_data_df[["Lat", "Lng"]]
humidity = weather_data_df["Humidity"]

In [5]:
fig = gmaps.figure(map_type="TERRAIN")
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the DataFrame to fit my ideal type of weather condition. My criteria is as follow:
# A max temperature lower or equal to 90 degrees and higher than 55 degrees.
# Wind speed is equal of less than 15 mph.
# humidity = 50
# cloudiness = 0
# created a new data frame that has these for criterias 


my_conditions_df = weather_data_df.loc[(weather_data_df["Wind Speed (mph)"] <= 10) & (weather_data_df["Cloudiness"] == 0) & \
                                   (weather_data_df["Max Temp"] >= 55) & (weather_data_df["Max Temp"] <= 90)].dropna()

my_conditions_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed (mph),Country,Date
14,Kwinana,-32.25,115.77,72.00,49,0,5.82,AU,11/22/2020
44,Iranshahr,27.20,60.68,61.66,76,0,5.59,IR,11/22/2020
53,Arrecife,28.96,-13.55,71.60,56,0,5.82,ES,11/22/2020
77,Abu Dhabi,24.47,54.37,75.20,50,0,3.36,AE,11/22/2020
81,Kenda,23.20,86.53,59.54,44,0,6.20,IN,11/22/2020
98,Kiryat Gat,31.61,34.76,63.00,100,0,1.99,IL,11/22/2020
117,Chake Chake,-5.25,39.77,78.08,84,0,6.26,TZ,11/22/2020
141,Faya,18.39,42.45,57.20,20,0,5.82,SA,11/22/2020
145,Busselton,-33.65,115.33,64.99,67,0,7.16,AU,11/22/2020
191,Rocha,-34.48,-54.33,58.77,83,0,7.81,UY,11/22/2020


In [7]:
len(my_conditions_df)

43

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
#create a new DataFrame for hotel
hotel_df = my_conditions_df.loc[:,["City","Country", "Lat", "Lng"]]

hotel_df["Hotel Name"] = ""

# Display the result
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
14,Kwinana,AU,-32.25,115.77,
44,Iranshahr,IR,27.20,60.68,
53,Arrecife,ES,28.96,-13.55,
77,Abu Dhabi,AE,24.47,54.37,
81,Kenda,IN,23.20,86.53,


In [9]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [10]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("No hotel found")
        
    print("------------")
    

Retrieving Results for Index 14: Kwinana.
Closest hotel in Kwinana is Hotel Clipper.
------------
Retrieving Results for Index 44: Iranshahr.
Closest hotel in Iranshahr is مهمانسرای طاها ایرانشهر.
------------
Retrieving Results for Index 53: Arrecife.
Closest hotel in Arrecife is Occidental Lanzarote Playa.
------------
Retrieving Results for Index 77: Abu Dhabi.
Closest hotel in Abu Dhabi is Royal M Hotel & Resort Abu Dhabi.
------------
Retrieving Results for Index 81: Kenda.
No hotel found
------------
Retrieving Results for Index 98: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 117: Chake Chake.
Closest hotel in Chake Chake is Hotel Archipelago.
------------
Retrieving Results for Index 141: Faya.
No hotel found
------------
Retrieving Results for Index 145: Busselton.
Closest hotel in Busselton is Abbey Beach Resort.
------------
Retrieving Results for Index 191: Rocha.
Closest hotel in Rocha is Casa Rodo 53.
------------
Retrie

In [11]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
14,Kwinana,AU,-32.25,115.77,Hotel Clipper
44,Iranshahr,IR,27.20,60.68,مهمانسرای طاها ایرانشهر
53,Arrecife,ES,28.96,-13.55,Occidental Lanzarote Playa
77,Abu Dhabi,AE,24.47,54.37,Royal M Hotel & Resort Abu Dhabi
81,Kenda,IN,23.20,86.53,


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))